# Prediction

## Description

Using the model we created in the `3_Train_fit_evaluate_classifier.ipynb`, this notebook will make predictions on new data to generate a cropland mask for Eastern Africa. The notebook will ceate both pixel-wise classifications and classification probabilities. Results are saved to disk as Cloud-Optimised-Geotiffs.

1. Open and inspect the shapefile which delineates the extent we're classifying
2. Import the model
3. Make predictions on new data loaded through the ODC.  The pixel classification will also undergo a post-processing step where steep slopes and water are masked using a SRTM derivative and WOfS, respectively. Pixels labelled as crop above 3600 metres ASL are also masked. 

***
## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load Packages

In [ ]:
import warnings

import os
import datacube
import numpy as np
import xarray as xr
import geopandas as gpd
from joblib import load
from tqdm.auto import tqdm
from datacube.utils import geometry
from datacube.utils.cog import write_cog
from datacube.utils.geometry import assign_crs
from datacube.testutils.io import rio_slurp_xarray

from deafrica_tools.classification import HiddenPrints, predict_xr
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.dask import create_local_dask_cluster

#import out feature layer function for prediction
from feature_layer_functions import gm_mads_two_seasons_prediction
from post_processing import post_processing

In [ ]:
warnings.filterwarnings("ignore")

### Set up dask cluster

In [ ]:
create_local_dask_cluster()

## Analysis parameters

* `model_path`: The path to the location where the model exported from the previous notebook is stored
* `training_data`: Name and location of the training data `.txt` file output from runnning `1_Extract_training_data.ipynb`
* `test_shapefile`: A shapefile containing polygons that represent regions where you want to test your model. The shapefile should have a unique identifier as this will be used to export classification results to disk as geotiffs. Alternatively, this could be a shapefile that defines the extent of the entire AOI you want to classify.
* `results`: A folder location to store the classified geotiffs
* `output_suffix`: A suffix to add to the exported classifications corresponding to the model iteration

In [ ]:
aoi = 'data/Area_of_interest.geojson'

output_suffix = 'area_1'

column = 'Id'

year = '2021'

results = 'results/classifications/'

model_path = 'data/southern_ml_model_20211108.joblib'

training_data = "data/southern_training_data_20211108.txt"

### Open and inspect AOI

In [ ]:
gdf = gpd.read_file(aoi)

In [ ]:
gdf.explore(column=column)

## Open the model


In [ ]:
model = load(model_path).set_params(n_jobs=1)

In [ ]:
# load the data
model_input = np.loadtxt(training_data)

# load the column_names
with open(training_data, 'r') as file:
    header = file.readline()
    
column_names = header.split()[1:][1:]

## Making a prediction


### Loop through tiles and predict, with post-processing

For every tile we list in the `aoi`, we calculate the feature layers, and then use the DE Africa function `predict_xr` to classify the data.

The `feature_layer_functions.gm_mads_two_seasons_prediction` function is doing most of the heavy-lifting here.

The results are exported to file as Cloud-Optimised Geotiffs.

In [ ]:
#make a folder to store results in if one doesn't already exist
if not os.path.exists(results+output_suffix):
        os.mkdir(results+output_suffix)

Handle dask garbage collections warnings

In [ ]:
import gc
g0, g1, g2 = gc.get_threshold()
gc.set_threshold(g0*2, g1*2, g2*2)

### Loop through polygons and predict the crop mask

In [ ]:
%%time
i=1
for index, row in gdf.iterrows():
    print('working on polygon: '+str(i)+"/"+str(len(gdf)),end='\r')
    
    # Get the geometry
    geom = geometry.Geometry(row.geometry.__geo_interface__,
                             geometry.CRS(f'EPSG:{gdf.crs.to_epsg()}'))
    
    #create query 
    query = {
        'geopolygon': geom,
        'time': year,
        'resolution': (-40,40),
        'output_crs': 'epsg:6933',
        'dask_chunks' : {'x':5000, 'y':5000},
    }
    
    # define features from query
    data = gm_mads_two_seasons_prediction(query)

    #generate prediction
    predicted = predict_xr(model,data,clean=True).compute()

    #-------Post-processsing ------------------------------       
    predict = post_processing(predicted)
    
    #create mask from polygon
    mask = xr_rasterize(gdf.iloc[[index]], data)
    predict = predict.where(mask)
    
    #----export classifications to disk-----------------------
    write_cog(predict,
              results+output_suffix+'/Southern_cropmask_'+output_suffix+"_"+year+'.tif',
              overwrite=True)
    i+=1

### Plotting

In [ ]:
import xarray as xr
predict=xr.open_rasterio(results+output_suffix+'/Southern_cropmask_'+output_suffix+"_"+year+'.tif').squeeze()

In [ ]:
predict.plot.imshow(size=12);

## Next steps

1. [Extracting_training_data](1_Extracting_training_data.ipynb) 
2. [Inspect_training_data](2_Inspect_training_data.ipynb)
3. [Train_fit_evaluate_classifier](3_Train_fit_evaluate_classifier.ipynb)
4. **Predict (this notebook)**


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

